In [9]:
# パッケージ
import pandas as pd
from datetime import datetime
pd.options.display.width = 100

In [10]:
# 2015/01~2023/04までを読み込みまとめる
# chart name
chart="USDJPY"
#column name
columns=["date","time","open","high","low","close","volume"]
# empty dataframe
all_chart_data = pd.DataFrame(index=[], columns=columns)

for year in range(2015, 2024):
    months = 12 if year==2024 else 4
    for j in range(months):
        # ファイル名
        file_path = f"../../charts/{chart}/one-minutes/{chart}_{str(year)}_{str(j+1).zfill(2)}.csv"
        csv_data = pd.read_csv(file_path, header=None, names=columns)
        all_chart_data = pd.concat([all_chart_data, csv_data], ignore_index=True)

print(all_chart_data)

               date   time     open     high      low    close volume
0        2015.01.02  00:00  119.743  119.748  119.741  119.748      0
1        2015.01.02  00:01  119.748  119.750  119.699  119.699      1
2        2015.01.02  00:02  119.699  119.706  119.699  119.704      0
3        2015.01.02  00:03  119.704  119.709  119.704  119.709      0
4        2015.01.02  00:04  119.709  119.709  119.701  119.701      1
...             ...    ...      ...      ...      ...      ...    ...
1095114  2023.04.28  23:53  136.305  136.307  136.250  136.270    161
1095115  2023.04.28  23:54  136.277  136.289  136.272  136.277    124
1095116  2023.04.28  23:55  136.273  136.296  136.272  136.284    237
1095117  2023.04.28  23:56  136.286  136.290  136.271  136.278    175
1095118  2023.04.28  23:57  136.279  136.314  136.278  136.299    147

[1095119 rows x 7 columns]


In [11]:
# １時間足を計算
# list for input result
result_60 = pd.DataFrame(index=[], columns=all_chart_data.columns.values)

# 時間が変わった最初の(index, hour)
hour_first_index = [-1, -1]
# 範囲内の高値と安値と出来高
high = -1
low = -1
volume = 0
print(all_chart_data.index)


for i in range(len(all_chart_data)):
    txt_datetime = f"{all_chart_data.iloc[i, 0]} {all_chart_data.iloc[i, 1]}"
    dt = datetime.strptime(txt_datetime, "%Y.%m.%d %H:%M")
    # 初期設定
    if hour_first_index[0] < 0:
        hour_first_index = [i, dt.hour]
        high = all_chart_data.iloc[i, 3]
        low = all_chart_data.iloc[i, 4]
        volume = 0
    
    # 時刻が変わったらリザルトに追加＆初期化
    if hour_first_index[1] != dt.hour:
        # add data
        new_row = pd.DataFrame([[
            all_chart_data.iloc[hour_first_index[0], 0],
            all_chart_data.iloc[hour_first_index[0], 1],
            all_chart_data.iloc[hour_first_index[0], 2],
            high,
            low,
            all_chart_data.iloc[i-1, 5],
            volume
            ]], columns=columns)
        print(new_row)
        result_60 = pd.concat([result_60, new_row], ignore_index=True )
        # init
        hour_first_index = [i, dt.hour]
        high = all_chart_data.iloc[i, 3]
        low = all_chart_data.iloc[i, 4]
        volume = 0
    else :
        # 大きいほうと小さい方を代入
        high = max(high, all_chart_data.iloc[i, 3])
        low = min(low, all_chart_data.iloc[i, 4])
        volume += all_chart_data.iloc[i, 6]
        
    # 最後にも更新
    if i == len(all_chart_data)-1:
        new_row = pd.DataFrame([[
            all_chart_data.iloc[hour_first_index[0], 0],
            all_chart_data.iloc[hour_first_index[0], 1],
            all_chart_data.iloc[hour_first_index[0], 2],
            high,
            low,
            all_chart_data.iloc[i, 5],
            volume
            ]], columns=columns)
        print(new_row)
        result_60 = pd.concat([result_60, new_row], ignore_index=True )

print(result_60)
    

RangeIndex(start=0, stop=1095119, step=1)
         date   time     open     high      low   close  volume
0  2015.01.02  00:00  119.743  119.843  119.699  119.82     179
         date   time    open     high      low    close  volume
0  2015.01.02  01:00  119.82  119.967  119.803  119.884     874
         date   time     open     high      low    close  volume
0  2015.01.02  02:00  119.884  120.073  119.836  120.071     601
         date   time     open     high      low    close  volume
0  2015.01.02  03:00  120.071  120.421  120.048  120.278    1088
         date   time     open     high     low    close  volume
0  2015.01.02  04:00  120.278  120.427  120.26  120.359     666
         date   time     open     high      low    close  volume
0  2015.01.02  05:00  120.359  120.372  120.274  120.326     449
         date   time     open     high      low    close  volume
0  2015.01.02  06:00  120.326  120.476  120.323  120.439     477
         date   time     open    high      low    clos

In [12]:
# 作成した結果を保存
# 無ければ保存先ディレクトリ作成
from pathlib import Path
dir_path = f"../../charts/{chart}/one-hours"
dir = Path(dir_path)
dir.mkdir(parents=True, exist_ok=True)

# 保存
save_path = f"{dir_path}/{chart}_all.csv"
result_60.to_csv(save_path, index=False)
# 保存
save_path = f"../charts/{chart}/one-minutes/{chart}_all.csv"
all_chart_data.to_csv(save_path, index=False)